## Simple Local Example

We'll do a simple local counting example, make sure to watch the video for this, the example will only work on Linux type systems, not on a Windows computer. This makes sense because you won't run this on Windows in the real world. Definitely watch the video for this one, a lot of it can't be replicated on Jupyter Notebook by itself!

In [1]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

In [3]:
#invoke once starts
ssc = StreamingContext(sc, 10 )  # socket object for Streaming   and 10 is batch duration in which streaming data didvide
sqlContext = SQLContext(sc)     #  object for Sql 

In [4]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5555)
#ssc have differnt streaming options hive,kafka,socket

In [5]:
#since it is Streaming so DRDD
#spark store data in DRDD format(ie ram storage ie mm storage)....so lines is Rdd object
lines = socket_stream.window( 20 )
#all streamed anlaysis  stores for 20 seconds 


In [6]:
# Now we got DRdd object...perform Sql,MLIB,DATAFRAME(transfomrmation,actions),Graphx

In [7]:
from collections import namedtuple # for creating tuple subclasses with named fields
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [8]:
#transformations

In [9]:
a=lines.flatMap( lambda text: text.split(" ") ) #Splits to a list

In [10]:
b=a.filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls

In [12]:
c=b.map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word

In [13]:
d=c.reduceByKey( lambda a, b: a + b ) # Reduces

In [14]:
e=d.map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object

In [22]:
f=e.foreachRDD( lambda rdd: rdd.toDF().sort(desc("count")) # Sorts Them in a DF
                                   #  ie Output Operations on DStreams
  .limit(10).createOrReplaceTempView("tweets1") )  # Registers to a table.

In [18]:
# Now run TweetRead.py
ssc.start()    

In [18]:
#!conda install -c anaconda seaborn -y  since new environment ie pyspark

In [19]:
import time
from IPython import display
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [20]:
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag,count from tweets1' )
    top_10_tweets.show()
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    sns.plt.show()
    count = count + 1

AnalysisException: Table or view not found: tweets1; line 1 pos 22;
'Project ['tag, 'count]
+- 'UnresolvedRelation [tweets1]


In [ ]:
ssc.stop()